In [2]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time
from PIL import Image
import cv2

from keras import backend as K
from tensorflow import keras
from tensorflow.keras.preprocessing import image
print(tf.__version__)

2.1.0


Using TensorFlow backend.


### im2col方法

In [54]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """

    Parameters
    ----------
    input_data : 由(数据量, 通道, 高, 长)的4维数组构成的输入数据
    filter_h : 滤波器的高
    filter_w : 滤波器的长
    stride : 步幅
    pad : 填充

    Returns
    -------
    col : 2维数组
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [55]:
sample = np.arange(25).astype("float32").reshape([1,1,5,5])

In [65]:
input = im2col(sample, 3, 3, stride=1, pad=0)

In [66]:
input

array([[ 0.,  1.,  2.,  5.,  6.,  7., 10., 11., 12.],
       [ 1.,  2.,  3.,  6.,  7.,  8., 11., 12., 13.],
       [ 2.,  3.,  4.,  7.,  8.,  9., 12., 13., 14.],
       [ 5.,  6.,  7., 10., 11., 12., 15., 16., 17.],
       [ 6.,  7.,  8., 11., 12., 13., 16., 17., 18.],
       [ 7.,  8.,  9., 12., 13., 14., 17., 18., 19.],
       [10., 11., 12., 15., 16., 17., 20., 21., 22.],
       [11., 12., 13., 16., 17., 18., 21., 22., 23.],
       [12., 13., 14., 17., 18., 19., 22., 23., 24.]])

In [67]:
kernal = np.array([1,-1,1,0,-1,0,0,1,0]).astype("float32").reshape(9,1)

In [68]:
out = np.dot(input,kernal)

In [60]:
out.reshape([3,3])

array([[ 6.,  7.,  8.],
       [11., 12., 13.],
       [16., 17., 18.]])

### unroll kernal

In [17]:
input_np = np.arange(25).astype("float32").reshape(-1,1)

In [18]:
input_np

array([[ 0.],
       [ 1.],
       [ 2.],
       [ 3.],
       [ 4.],
       [ 5.],
       [ 6.],
       [ 7.],
       [ 8.],
       [ 9.],
       [10.],
       [11.],
       [12.],
       [13.],
       [14.],
       [15.],
       [16.],
       [17.],
       [18.],
       [19.],
       [20.],
       [21.],
       [22.],
       [23.],
       [24.]], dtype=float32)

In [19]:
kernal_np = np.array([1,-1,1,0,-1,0,0,1,0]).astype("float32").reshape(3,3)

In [49]:
kernal_np 

array([[ 1., -1.,  1.],
       [ 0., -1.,  0.],
       [ 0.,  1.,  0.]], dtype=float32)

In [50]:
def unroll_kernel(kernel, n):

    m = kernel.shape[0]
    unrolled_K = np.zeros(((n - m + 1)**2, n**2))

    skipped = 0
    for i in range(n ** 2):
         if (i % n) < n - m + 1 and((i / n) % n) < n - m + 1:
             for j in range(m):
                 for l in range(m):
                    unrolled_K[i - skipped, i + j * n + l] = kernel[j, l]
         else:
             skipped += 1
    return unrolled_K

In [51]:
unroll_k = unroll_kernel(kernal_np,5)
unroll_k

array([[ 1., -1.,  1.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1., -1.,  1.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1., -1.,  1.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1., -1.,  1.,  0.,  0.,  0., -1.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1., -1.,  1.,  0.,  0.,  0., -1.,
         0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -1.,  1.,  0.,  0.,  0.,
        -1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., -1.,  1.,
         0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
      

In [52]:
result = np.dot(unroll_k,input_np)
result

array([[ 6.],
       [ 7.],
       [ 8.],
       [11.],
       [12.],
       [13.],
       [16.],
       [17.],
       [18.]])

In [53]:
np.dot(unroll_k.T,result).reshape([5,5])

array([[ 6.,  1.,  7., -1.,  8.],
       [11., -5.,  5., -9., 13.],
       [16., -4., 12., -6., 18.],
       [ 0., -5., -5., -5.,  0.],
       [ 0., 16., 17., 18.,  0.]])

### tf.nn.conv2d

In [25]:
input_tf = tf.constant(input_np,shape=[1,5,5,1])
input_tf[0,:,:,0]

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[ 0.,  1.,  2.,  3.,  4.],
       [ 5.,  6.,  7.,  8.,  9.],
       [10., 11., 12., 13., 14.],
       [15., 16., 17., 18., 19.],
       [20., 21., 22., 23., 24.]], dtype=float32)>

In [26]:
kernal_tf = tf.constant(kernal_np,shape=[3,3,1,1])

In [27]:
value = tf.nn.conv2d(input_tf, kernal_tf, strides=[1, 1, 1, 1], padding='VALID')

In [28]:
value

<tf.Tensor: shape=(1, 3, 3, 1), dtype=float32, numpy=
array([[[[ 6.],
         [ 7.],
         [ 8.]],

        [[11.],
         [12.],
         [13.]],

        [[16.],
         [17.],
         [18.]]]], dtype=float32)>

In [29]:
result = tf.nn.conv2d_transpose(value, kernal_tf, output_shape=[1,5,5,1], strides=[1, 1, 1, 1], padding='VALID')

In [30]:
result[0,:,:,0].numpy

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[ 6.,  1.,  7., -1.,  8.],
       [11., -5.,  5., -9., 13.],
       [16., -4., 12., -6., 18.],
       [ 0., -5., -5., -5.,  0.],
       [ 0., 16., 17., 18.,  0.]], dtype=float32)>>

### tf.keras.layers.Conv2D

In [31]:
def kernel_init(shape, dtype=None):
    b = K.constant([[[ 1., -1.,  1.],
       [ 0., -1.,  0.],
       [ 0.,  1.,  0.]]],shape=[3,3,1,1])
    return b

In [32]:
input_tf

<tf.Tensor: shape=(1, 5, 5, 1), dtype=float32, numpy=
array([[[[ 0.],
         [ 1.],
         [ 2.],
         [ 3.],
         [ 4.]],

        [[ 5.],
         [ 6.],
         [ 7.],
         [ 8.],
         [ 9.]],

        [[10.],
         [11.],
         [12.],
         [13.],
         [14.]],

        [[15.],
         [16.],
         [17.],
         [18.],
         [19.]],

        [[20.],
         [21.],
         [22.],
         [23.],
         [24.]]]], dtype=float32)>

In [33]:
conv = tf.keras.layers.Conv2D(filters=1,kernel_size=3,strides=(1,1),padding='valid',kernel_initializer=kernel_init)(input_tf)
conv

<tf.Tensor: shape=(1, 3, 3, 1), dtype=float32, numpy=
array([[[[ 6.],
         [ 7.],
         [ 8.]],

        [[11.],
         [12.],
         [13.]],

        [[16.],
         [17.],
         [18.]]]], dtype=float32)>

In [34]:
result = tf.keras.layers.Conv2DTranspose(filters=1,kernel_size=3,strides=(1,1),padding='valid',kernel_initializer=kernel_init)(conv)

In [35]:
result[0,:,:,0].numpy

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[ 6.,  1.,  7., -1.,  8.],
       [11., -5.,  5., -9., 13.],
       [16., -4., 12., -6., 18.],
       [ 0., -5., -5., -5.,  0.],
       [ 0., 16., 17., 18.,  0.]], dtype=float32)>>